In [1]:
from random import random, seed, randrange
from math import floor, pi
import matplotlib.pyplot as plt
import numpy as np
import os
from time import time
import pandas as pd
from scipy.integrate import simps, trapz, cumtrapz
from scipy.ndimage.filters import gaussian_filter1d
from time import time
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 9]

In [2]:
%run -i "../../carbon_walls/gcmc.py"

In [3]:
filename = "../data/" + "y0.75rep1" + ".csv"
data_dump = pd.read_csv(filename).copy()

for i in range(data_dump.shape[0]):
    if np.isnan( data_dump.yco_sim[i]):
        yco = data_dump.yco_res[i]
        P_res = data_dump.P_res[i]*10**5 #[Pa]
        T = data_dump.T_res[i] + 273.15 #K
        fco, fme = PR_Fugacity( P_res/10**5, T, yco )
        fco = fco*10**5
        fme = fme*10**5
        
        sf = False
        mega_verbose = False

        s_box = 57.15
        N_max = 50000
        Vol = s_box**3
        kb = 1.3806*10**(7) #[Pa*A^3/K]
        Nco = 0 #floor(fco*Vol/kb/T)
        Nme = 0 #floor(fme*Vol/kb/T)
        Nc = 0
        rc = s_box
        beta = 1/T
        zz_co = beta*fco
        zz_me = beta*fme
        delta = 1
        pi_move = 0.5

        N_moves = 100
        N_equil = int( np.round( 200000/N_moves) )
        N_prod = int( np.round( 100000/N_moves) )

        t0 = time()
        rhocov,rhomev,Env,Pv,Ncov, Nmev = mc_run()
        t = time()
        
        Nco = Ncov.mean()
        Nme = Nmev.mean()
        data_dump.yco_sim[i] = Nco/(Nco + Nme)
        data_dump.P_sim[i] = Pv.mean()*10 #[bar]
        data_dump.E_sim[i] = Env.mean()
        data_dump.rhoco[i] = rhocov.mean()
        data_dump.rhome[i] = rhomev.mean()
        data_dump.yco_s[i] = (rhocov/(rhocov + rhomev)).std()
        data_dump.P_s[i] = Pv.std()*10 #[bar]
        data_dump.rhoco_s[i] = rhocov.std()
        data_dump.rhome_s[i] = rhomev.std()
        data_dump.E_s[i] = Env.std()
        data_dump.time[i] = t - t0
        print( i, data_dump.yco_sim[i],  data_dump.P_sim[i], data_dump.rhoco[i], data_dump.rhome[i], data_dump.time[i] )
        #filename = "../data_output/yco" + str(yco) + "P" + str(P) + "/run" + str(i) + ".csv"
        data_dump.to_csv( filename, index = False)

0 0.7534268933117726 9.741167736270665 0.00018226717204760246 5.9650356601297556e-05 31.78064799308777
1 0.7388343768786984 19.715632517971272 0.00037179978921270395 0.00013142502117500482 62.5341420173645
2 0.7460266488981999 28.748406452352192 0.000567674188068638 0.00019325598635211617 97.25330662727356
3 0.7474635129529937 38.76182533731156 0.0008106674757574697 0.0002738904480865678 138.47454047203064
4 0.7408324834441381 48.575079129676375 0.0010461388612309861 0.0003659737075476873 172.78073000907898
5 0.7322125940245311 55.72128876284698 0.0012572878598334884 0.00045981980820447175 215.0924825668335
6 0.7185535815835044 64.42056672047968 0.0014960885266704271 0.0005859949323994433 259.0802114009857
7 0.7189548986362179 72.02340840042419 0.0017800681266333424 0.0006958425737596105 314.86544036865234
8 0.7146026735901155 80.815540845849 0.00214943524321232 0.0008584393739000151 368.30420207977295
9 0.7178088281125705 93.42107887160597 0.002876632817845526 0.0011308866012307702 44